In [2]:
import os
import json
import torch
import logging
import pandas as pd

C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [ ]:
# class CustomImageDataset(Dataset):


    
#     def __init__(self, , , transform=None, target_transform=None):
#         self.img_labels = pd.read_csv(, names=['file_name', 'label'])
#         self.img_dir = 
#         self.transform = transform
#         self.target_transform = target_transform

#     def __len__(self):
#         return len(self.img_labels)

#     def __getitem__(self, idx):
#         img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
#         image = (img_path)
#         label = self.img_labels.iloc[idx, 1]
#         if self.transform:
#             image = self.transform(image)
#         if self.target_transform:
#             label = self.target_transform(label)
#         return image, label

In [3]:
logger = logging.getLogger(__name__)

In [5]:
class InputExample(object):
    def __init__(self, guid, words, labels):
        self.guid = guid
        self.words = words
        self.lables = labels
    
    def __repr(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serialize this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serialize this instance to a Json string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + '\n'
        

In [8]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, word_ids, char_ids, mask, label_ids):
        self.word_ids = word_ids
        self.char_ids = char_ids
        self.mask = mask
        self.label_ids = label_ids
    
    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python diction"""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dump(self.to_dict(), intent=2, sort_keys=True) + "\n"

SyntaxError: unexpected EOF while parsing (<ipython-input-8-88955feb57a7>, line 13)

In [ ]:
class NaverNerProcessor(object):
    """Processor for thr Naver NER data set"""

    def __init__(self, args)):
        self.args = args
        self.labels_lst = get_labels(args)

    @classmethod
    def _read_file(cls, input_file):
        """Read tsv file, and return words and label as list"""
        with open(input_file, 'r', encoding='utf-8') as f:
            lines = []
            for line in f:
                lines.append(line.strip())
            return lines
    
    def _create_examples(self, dataset, set_type):
        """Create examples for thr training and dev sets"""
        examples = []
        for (i, data) in enumerate(dataset):
            words, labels = data.split('\t')
            words = words.split()
            labels = labels.split()
            guid = "%s-%s" % (set_type, i)

            assert len(words) == len(labels)

            if i % 10000 == 0:
                logger.info(data)
            examples.append(InputExample(guid = guid, words = words, labels = labels))
        return examples

    def get_examples(self, mode):
        """
        Args:
            mode: trina, dev, test
        """

        file_to_read = None
        if mode == "train":
            file_to_read = self.args.train_file
        elif mode == "dev":
            file_to_read = self.args.dev_file
        elif mode == "test":
            file_to_read = self.args.test_file

        logger.info(f"LOOKING AT {(os.path.join(self.args.data_dif, file_to_read))}")
        return self._create_examples(self._read_file(os.path.join(self.args.data_dir, file_to_read)), mode)
            

In [15]:
def load_word_matrix(args, word_vocab):
    if not os.path.exists(args.wordvec_dir):
        os.mkdir(args.wordvec_dir)

    # Making new word vector (as list type)
    logger.info("Building word matrix...")
    embedding_index = dict()
    with open(os.path.join(args.wordvec_dir, args.w2v_file) , 'r', encoding='utf-8', errors = 'ignore') as f:
        for line in f:
            tokens = line.rstrip().split(' ')
            embedding_index[tokens[0]] = list(map(float, tokens[1:]))

    word_matrix = np.zeros((args.word_vocab_size, args.word_emb_dim), dtype= 'float32')
    cnt = 0

    for word, i  in word_vocab.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            word_matrix[i] = np.asarray(embedding_vector, dtype='float32')
        else:
            word_matrix[i] = np.random.uniform(-0.25, 0.25, args.word_emb_dim)
            cnt += 1
    logger.info(f"{cnt} words not in pretrined matrix")

    word_matrix = torch.from_numpy(word_matrix)
    return word_matrix
